<a href="https://colab.research.google.com/github/haruka-inb/pytorch_practice/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# define device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define hyoer parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 100
learning_rate = 0.01

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root="/../../data", train=True,
                                      download=True, transform=transform)
testset = torchvision.datasets.MNIST(root="/../../data", train=False,
                                     download=True, transform=transform)

# create Data loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)


In [ ]:
# define RNN (many-to-one) class
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes=10):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  # forward method
  def forward(self, x):
    # set initial hidden status (short-term memory) and cell status (long-term memory)
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # forward propagate LSTM
    out, _ = self.lstm(x, (h0,c0))

    # decide the hidden state of the last time step
    out = self.fc(out[:,-1,:]) # because output from LSTM is (batch_size, seq_len, hidden_size)

    return out

# batch_first:
# If your input data is of shape (batch_size, seq_len, features)
# then you need batch_first=True and your LSTM will give output of shape (batch_size, seq_len, hidden_size).
# Without batch_first=True it will use the first dimension as the sequence dimension.
# With batch_first=True it will use the second dimension as the sequence dimension.

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
model

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train the model
for e in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.view(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)

  # forward pass
  outputs = model(images)
  loss = criterion(outputs, labels)

  # backward and optimize
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (i+1) % 100 == 0:
    print("Epochs {}/{}, Steps {}/{}: Training loss is {}"
    .format(e+1, num_epochs, i+1, len(train_loader), loss.item()))

# test the model
model.eval()
with torch.no_grad():
  correct, total = 0, 0
  for images, labels in test_loader:
    images = images.view(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, pred = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (pred==labels).sum().item()

  print("Test Accuracy of the model on the 10000 test images: {} %"
  .format(100*correct/total))

# save the model checkpoint
torch.save(model.state_dict(), "model.ckpt")

Epochs 1/100, Steps 600/600: Training loss is 2.3019378185272217
Epochs 2/100, Steps 600/600: Training loss is 2.3254575729370117
Epochs 3/100, Steps 600/600: Training loss is 2.287538528442383
Epochs 4/100, Steps 600/600: Training loss is 2.2600953578948975
Epochs 5/100, Steps 600/600: Training loss is 2.176805019378662
Epochs 6/100, Steps 600/600: Training loss is 2.4118549823760986
Epochs 7/100, Steps 600/600: Training loss is 2.107081890106201
Epochs 8/100, Steps 600/600: Training loss is 2.0793709754943848
Epochs 9/100, Steps 600/600: Training loss is 2.071587324142456
Epochs 10/100, Steps 600/600: Training loss is 2.113895893096924
Epochs 11/100, Steps 600/600: Training loss is 2.0474538803100586
Epochs 12/100, Steps 600/600: Training loss is 1.8463839292526245
Epochs 13/100, Steps 600/600: Training loss is 1.7465368509292603
Epochs 14/100, Steps 600/600: Training loss is 1.7915867567062378
Epochs 15/100, Steps 600/600: Training loss is 1.8037896156311035
Epochs 16/100, Steps 600